<a href="https://colab.research.google.com/github/vpaliwal1/Image-Colorization-using-Encoder-Decoder/blob/master/Image_colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Data_Science/Colorization"

 10090824183_d02c613f10_m.jpg	     4625089819_55c45a189c.jpg
 102501987_3cdb8e5394_n.jpg	     4644336779_acd973528c.jpg
 10503217854_e66a804309.jpg	     4648680921_80dfc4f12a.jpg
 10894627425_ec76bbc757_n.jpg	     4654893119_45d232016b.jpg
 110472418_87b6a3aa98_m.jpg	     466486216_ab13b55763.jpg
 11102341464_508d558dfc_n.jpg	     4675532860_890504a4a3_m.jpg
 11233672494_d8bf0a3dbf_n.jpg	     4684127262_6c3346188d.jpg
 11694025703_9a906fedc1_n.jpg	     4703837996_d3c1df77cc_n.jpg
 118974357_0faa23cce9_n.jpg	     4713531680_1110a2fa07_n.jpg
 11944957684_2cc806276e.jpg	     4713533500_fcc295de70_n.jpg
 12165480946_c4a3fe182d_n.jpg	     4735314389_94fe1b2a9f_n.jpg
 12202373204_34fb07205b.jpg	     475936554_a2b38aaa8e.jpg
 12238827553_cf427bfd51_n.jpg	     475947979_554062a608_m.jpg
 12240165555_98625b1e88_n.jpg	     4764674741_82b8f93359_n.jpg
 12240303_80d87f77a3_n.jpg	     4765063233_f64440c20b.jpg
 12240577184_b0de0e53ea_n.jpg	     4797595918_79887b1229_n.jpg
 12243068283_ee4c2683e2_n.

In [0]:
path = "/content/drive/My Drive/Data_Science"

In [0]:
import tensorflow
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np

In [0]:
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
#Resize images, if needed
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=738, 
                                          class_mode=None)

Found 738 images belonging to 1 classes.


In [0]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(738, 256, 256, 1)
(738, 256, 256, 2)


In [0]:
#Encoder

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 32, 32, 512)      

In [0]:
model.fit(X,Y,validation_split=0.1, epochs=300, batch_size=32)

Epoch 1/300
21/21 [==============================] - 4s 191ms/step - loss: 0.0281 - accuracy: 0.5867 - val_loss: 0.0263 - val_accuracy: 0.5794
Epoch 2/300
21/21 [==============================] - 4s 192ms/step - loss: 0.0284 - accuracy: 0.5910 - val_loss: 0.0248 - val_accuracy: 0.6014
Epoch 3/300
21/21 [==============================] - 4s 191ms/step - loss: 0.0280 - accuracy: 0.5919 - val_loss: 0.0255 - val_accuracy: 0.5977
Epoch 4/300
21/21 [==============================] - 4s 190ms/step - loss: 0.0287 - accuracy: 0.5871 - val_loss: 0.0255 - val_accuracy: 0.5966
Epoch 5/300
21/21 [==============================] - 4s 190ms/step - loss: 0.0281 - accuracy: 0.5955 - val_loss: 0.0248 - val_accuracy: 0.5985
Epoch 6/300
21/21 [==============================] - 4s 192ms/step - loss: 0.0278 - accuracy: 0.5960 - val_loss: 0.0243 - val_accuracy: 0.6022
Epoch 7/300
21/21 [==============================] - 4s 191ms/step - loss: 0.0276 - accuracy: 0.5955 - val_loss: 0.0263 - val_accuracy: 0.5938

In [0]:
model.save('colorize_autoencoder300.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: colorize_autoencoder300.model/assets


In [0]:
model_save_name = 'colorize_autoencoder300.model'
path = F"/content/drive/My Drive/Data_Science/{model_save_name}" 
model.save(model_save_name, path)

INFO:tensorflow:Assets written to: colorize_autoencoder300.model/assets


In [0]:
from google.colab import files

In [0]:
files.upload()

Saving download.jpg to download (3).jpg


{'download.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc2\x00\x11\x08\x02\x80\x01\xe0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\xff\xc4\x00\x14\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x01\xf2\x90\x00\x00T\x9c\x93wJ\x81\xbf&F\xa1\x95\xc4\xf6\xbcP=\x9a\x86Y,@\x00\x00\x00\n*\x08\x00\x12<\x80\x00\x00\x00\x00Q\x14\x00\x00G \x8a\x00\xa8\xa0\xe4R0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05@s@],\xc0\xd5\xca\x00\xe9\xf9\x89\x8b\xf9Vk\x88\x00\x00\x00\n\x80\n\x8ahk

In [0]:
img1_color=[]
img1=img_to_array(load_img('download (3).jpg'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
imsave("result.png", lab2rgb(result))

/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 54 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [0]:
files.download("result.png")